In [13]:
# Packages
import pandas as pd
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

from bokeh.io import output_notebook#, show
from bokeh.models import DataRange1d
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show
output_notebook()

def plot_config(width: int, height: int,title=None,
                x_label: str = None, 
                y_label: str = None):
    p = figure(title=title, background_fill_color="#fafafa", width=width, height=height)
    p.output_backend = "svg"    # save plot in HTML as SVG
    p.title.text_font_size = "12pt"
    p.xaxis.axis_label = x_label
    p.yaxis.axis_label = y_label
    p.yaxis.axis_label_text_font_size = "12pt"
    p.yaxis.major_label_text_font_size = "12pt"
    p.xaxis.major_label_text_font_size = "12pt"
    p.xaxis.axis_label_text_font_size = "12pt"
    return p

Loading BokehJS ...

# Load Sim Result

In [2]:
file_name = 'KF'
# file_name = 'KF_nodelay'


df_sim = pd.read_csv('./sim_result/' + file_name + '.csv')
columns = df_sim.columns.to_list()
t0 = df_sim['time'][0]

cond1 = df_sim['time'] - t0 >= 10
cond2 = df_sim['time'] - t0 <= 20
df_sim = df_sim[cond1*cond2]

gpr_t0 = 10
gpr_tn = 20

df_sim.drop('Unnamed: 0', axis=1, inplace=True)

df_tf = df_sim[df_sim['sensor.type'] == 'tf']


# Euclidean Error based on GPR of TF Data

In [25]:
cir_size = 2
p_total = []


# ---------------------------------------------------

ti = np.array(df_tf['time'] - t0).reshape(-1,1)
xi = np.array(df_tf['tf.x']).reshape(-1)
yi = np.array(df_tf['tf.y']).reshape(-1)

kernel = 3 * RBF(length_scale=0.7, length_scale_bounds='fixed')
gpr_x = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=0)
gpr_x.fit(ti, xi)
gpr_y = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=0)
gpr_y.fit(ti, yi)

condi1 = df_sim['time'] - t0 >= gpr_t0
condi2 = df_sim['time'] - t0 <= gpr_tn
df_window = df_sim[condi1*condi2]

t_gpr = df_window['time'].to_numpy().reshape(-1,1) - t0
x_gpr = gpr_x.predict(t_gpr)
y_gpr = gpr_y.predict(t_gpr)
t_gpr = t_gpr.reshape(-1)

df_win_rt = df_window[-df_window['rt_x'].isna()]

t_rt_gpr = df_win_rt['time'].to_numpy().reshape(-1,1) - t0
x_rt_gpr = gpr_x.predict(t_rt_gpr)
y_rt_gpr = gpr_y.predict(t_rt_gpr)
t_rt_gpr = t_rt_gpr.reshape(-1)

# Mean of Absolute Error (MAE) with GPR
dx_rt = df_win_rt['rt_x'].to_numpy() - x_rt_gpr
dy_rt = df_win_rt['rt_y'].to_numpy() - y_rt_gpr
dx = df_window['x'].to_numpy() - x_gpr
dy = df_window['y'].to_numpy() - y_gpr
norm2 = np.sqrt(dx ** 2 + dy ** 2)
norm2_rt = np.sqrt(dx_rt ** 2 + dy_rt ** 2)
mae_offline = np.average(norm2)
mae_rt = np.average(norm2_rt)

p2 = plot_config(width=600, height=600, title='Euclidean Error of Sim based on GPR', x_label='x [ m ]', y_label='y [ m ] ')
p2.match_aspect=True
p2.aspect_scale=1
p2.circle(x=df_sim['rt_x'], y=df_sim['rt_y'], legend_label='Sim', size=cir_size, fill_color='royalblue', line_color='royalblue')
p2.circle(x=x_rt_gpr, y=y_rt_gpr, legend_label='GPR', size=cir_size, fill_color='green',line_color='green')
p2.circle(x=df_tf['tf.x'], y=df_tf['tf.y'], legend_label='CAM', size=cir_size, fill_color='black',line_color='black')
p2.circle(x=df_sim['rt_x'].iloc[0], y=df_sim['rt_y'].iloc[0], legend_label=f'Start', size=10, fill_color='crimson', line_color='crimson')
p2.square(x=df_sim['rt_x'].iloc[-1], y=df_sim['rt_y'].iloc[-1], legend_label=f'End', size=10, fill_color='crimson', line_color='crimson')
# p2.legend.nrows=1
p2.legend.ncols=1
p2.legend.location = "bottom_left"
p_total.append(p2)


p20 = plot_config(width=600, height=400, title="Euclidean Error of Sim based on GPR", x_label='t [ s ]', y_label='||SIM - GPR|| [ m ] ')
p20.circle(x=t_rt_gpr, y=norm2_rt, legend_label='Sim', size=cir_size, fill_color='royalblue', line_color='royalblue')
p20.line(x=[t_rt_gpr[0], t_rt_gpr[-1]], y=[mae_rt]*2, legend_label=f'avg: {mae_rt:.3f}, std: {np.std(norm2_rt):.3f}', line_width=2, line_dash=[6,3], line_color='darkblue')
p20.legend.ncols=1
p20.legend.location = "top_left"
p_total.append(p20)

for i in p_total:
    show(i, notebook_handle=True)